In [3]:
from humanfriendly import prompts
import sys
from pathlib import Path
import logging
import os
from rich.console import Console
from rich.markdown import Markdown
from rich.measure import Measurement
from rich.table import Table
import constants
import ArgConfigParse

In [4]:
logging.getLogger(__name__)
logging.root.setLevel(logging.DEBUG)

In [5]:
logging.root.setLevel(logging.INFO)

In [6]:
def sounds(voice='hiss', repeat=5):
    for i in range(0, repeat):
        print(voice)
    return True

In [7]:
class Menu:
    def __init__(self):
        self.menu = {}

    def add_function(self, item_name, function, args=[], kwargs={}):
        self.menu[item_name] = {'function': function, 'args': args, 'kwargs': kwargs}
    
    def get_item(self, choice):
        if isinstance(self.menu[choice], dict) and 'function' in self.menu[choice].keys():
            val = self._execute(choice)
            return choice, val
        if choice not in self.menu.keys():
            return False, None
        if choice in self.menu.keys():
            return choice, self.menu[choice]
    
    def add_item(self, item_name, return_value=None):
        if return_value:
            self.menu[item_name] = return_value
        else:
            self.menu[item_name] = item_name
            
    def add_return(self, item_name='← Return to previous menu'):
        self.add_item(item_name, return_value='_cancel')
    
    def add_quit(self, item_name='[X] Quit [X]', message='User exit'):
        self.add_function(item_name, doExit, [message, 0])
    
    def _execute(self, choice):
        func = self.menu[choice]['function']
        val = func(*self.menu[choice]['args'], **self.menu[choice]['kwargs'])
        return val
        
        

In [8]:
def doExit(message=None, exit=0):
    logging.debug('doExit')
    if not message:
        message = ''
    if exit > 0:
        logging.warning(message)
        print(f'\nExit: {message}')
        sys.exit(exit)
    else:
        print(f'\n{message}')
        sys.exit(exit)

In [9]:
def get_google_drive(config): #entryPoint='/Volumes/GoogleDrive/Shared drives/'):

    if not isinstance(config, dict):
        doExit(f'TypeError: config must be of type `dict`')
    menu = Menu()
    keys = ['google_drive_entry', 'google_drive']
    
    try:
        google_drive_entry = Path(config['google_drive_entry']).resolve()
    except KeyError as e:
        google_drive_entry = Path('/Volumes/GoogleDrive/Shared drives/').resolve()
        config['google_drive_entry'] = google_drive_entry
        logging.warning(f'"google_drive_entry" missing from config file; set to {google_drive_entry}')
    
    try:
        google_drive = config['google_drive']
    except KeyError as e:
        logging.info('google_drive not set')
        google_drive = None
 
    
    MARKDOWN='''
# Google Drive - Shared Drive Selection

## Instructions:
Choose the shared drive that contains the student cumulative folders.
Select a Shared Drives by entering the number or typing the name

Google File Stream must be running and a user that has write-access to the student folders must be signed in.

For help installing and configuring google filestream please see this link: [Google File Stream Support](https://support.google.com/a/answer/7491144?utm_medium=et&utm_source=aboutdrive&utm_content=getstarted&utm_campaign=en_us)
'''
    
    # check if entry point is valid
    if not google_drive_entry.is_dir():
        print('Google Filestream does not appear to be available.')
        print('Google Filestream is required for this applicaiton.')
        print('Please install google filestream and sign in before running this application')
        doExit('Filestream not running', 0)
    
    # get a list of dirs
    dirs = next(os.walk(google_drive_entry))[1]
    
    writeable_dirs = {}
    # get a list of writeable dirs
    for d in dirs:
        if os.access(google_drive_entry/d, os.W_OK):
            writeable_dirs[d] = google_drive_entry/d
    
    # build a menu from the google drives that are writeable
    for key in sorted(writeable_dirs.keys()):
        menu.add_item(key, writeable_dirs[key])
    
    menu.add_return()
    menu.add_quit()    

    # display the header text
    console = Console()
    md = Markdown(MARKDOWN)
    console.print(md)    

    # display the values
    table = Table(title='Google Drive Settings')
    table.add_column('Setting', style='blue', no_wrap=True)
    table.add_column('Current Setting', style='magenta')
    table.add_row(f'Cumulative Folders', f'{google_drive}' )
    table.add_row(f'FileStream Mount', f'{google_drive_entry}')
    console.print(table)
    
    
    # get user input
    choice = prompts.prompt_for_choice(menu.menu)
    
    # set values as needed
    config['google_drive'] = google_drive_entry/choice
    config['google_drive_entry'] = google_drive_entry
    
    
    # return the choice and the modified config dict
    return {'settings': config}
    

In [10]:
def choose_new_dir():
    
    dirs = ['Desktop', 'Documents', 'Downloads']
    menu = Menu()
    MARKDOWN = f'''
# Choose a New Directory
Choose one of the following directories to search for Student Exports
'''
    md = Markdown(MARKDOWN)
    console = Console()
    console.print(md)           

    b = Path('~/').expanduser()
    
    for d in dirs:
        d = b/Path(d)
        print(d.name, d)
        menu.add_item(str(d.name), d)
    
    menu.add_return()
    menu.add_quit()
    
    choice = prompts.prompt_for_choice(menu.menu)
    return menu.get_item(choice)
        

In [11]:
def choose_student_export(config={}):
    last_folder = Path('~/').expanduser()
    try:
        last_folder = Path(config[last_folder])
    except KeyError:
        last_folder = Path('~/Downloads').expanduser()
    
    file_glob = sorted(last_folder.glob('*'))
    
    menu = Menu()
    for f in file_glob:
        if f.is_file():
            menu.add_item(f.name, f)
    menu.add_return()
    menu.add_quit()
    
    choice = prompts.prompt_for_choice(menu.menu)
    
    print('#'*40)
    print(config)
    print('#'*40)
    
    config['__runtime']['student_export'] = menu.get_item(choice)[1]
    return config
#     return menu.get_item(choice['__runtime']={student_folder})
        
    
    

In [45]:
def main_menu(config={}):
    menu = Menu()
       
    menu.add_function('Set Cummulitive Folder', get_google_drive, kwargs={'config': config['settings']})
    menu.add_function('Choose Student Export', choose_student_export, kwargs={'config': config})
    menu.add_item('Quit', '_quit' )
    
    choice = prompts.prompt_for_choice(menu.menu)
    
    # get the name or the return value from function
    subChoice, values = menu.get_item(choice) 
    
    if values == '_quit':
        return '_quit', config
    
    if values:
        newConfig = ArgConfigParse.merge_dict(config, values)
    else:
        newConfig = config
    

    
    return subChoice, newConfig
        

In [46]:
def main():    
    appName = constants.appName
    develName = constants.develName
    userPath = constants.userPath
    version = constants.version
    configFile = constants.configFile

    baseConfig = Path(configFile).resolve()
    userConfig = Path('~/.config/').expanduser()/userPath/configFile


    configParser = ArgConfigParse.ConfigFile([baseConfig, userConfig])
    config = configParser.parse_config()
    config['__runtime'] = {}
    config['__runtime']['student_export'] = None
    

    logging.root.setLevel(config['base_config']['log_level'])
    
    
    MARKDOWN=f'''
# portfolioCreator Version: {version}
    
Create student Cumulative Folders in Google Drive using data from PowerSchool Learning
    
    '''
    
    # display the header text
    md = Markdown(MARKDOWN)
    
    
    while True:
        console = Console()
        console.print(md)    


        # display the values
        table = Table(title='Current Settings')
        table.add_column('Setting', style='blue', no_wrap=True)
        table.add_column('Current Setting', style='magenta')
        table.add_row(f'Cumulative Folder Location', f'{config["settings"]["google_drive"]}' )
        table.add_row(f'Debug Level', f'{config["base_config"]["log_level"]}' )                  
        table.add_row(f'Student Export File', f'{config["__runtime"]["student_export"]}')
        console.print(table)    
    
        choice, values = main_menu(config)
        if choice == '_cancel':
            logging.debug('_cancel')
        
        if choice == '_quit':
            ArgConfigParse.write(config, userConfig)
            break
    
        print(config)
                      
    
    print('add cleanup here')    
    return config

In [47]:
f = main()



  1. Set Cummulitive Folder
  2. Choose Student Export
  3. Quit
 
 Enter your choice as a number or unique substring (Control-C aborts): 3
add cleanup here


In [ ]:
f